In [1]:
import os
import sys
os.chdir("/home/group2/naTtahN_T2/Tan3010/211023_NhatTan_Test")
sys.path.append("..")

In [2]:
from utilFuncs.modelUtils import *

Current directory: /home/group2/naTtahN_T2/Tan3010/211023_NhatTan_Test
Using device: cuda


In [3]:
# dataset = load_dataset('alt')
# train_dataset = dataset['train']
# test_dataset = dataset['test']
# train_dataset[0]

In [4]:
filesTrain = ['split_train2023_0to10k.vi', 'split_train2023_0to10k.lo', 
              'split_train2023_10kto30k.vi', 'split_train2023_10kto30k.lo',
              'split_train2023_40kto100k.vi', 'split_train2023_40kto100k.lo']
allPathsTrain = findFiles(filesTrain, '../**')

Found 6 paths


In [5]:
allDataTrain = readAllData(allPathsTrain)
train_dataset = createDataset([allDataTrain[0], allDataTrain[2], allDataTrain[4]],
                              [allDataTrain[1], allDataTrain[3], allDataTrain[5]])

Read 6 files


In [6]:
filesEval = ['split_val2023_0to10k.vi', 'split_val2023_0to10k.lo', 
              'split_val2023_10kto30k.vi', 'split_val2023_10kto30k.lo',
              'split_val2023_40kto100k.vi', 'split_val2023_40kto100k.lo']
allPathsEval = findFiles(filesEval, '../**')

Found 6 paths


In [7]:
allDataEval = readAllData(allPathsEval)
eval_dataset = createDataset([allDataEval[0], allDataEval[2], allDataEval[4]],
                             [allDataEval[1], allDataEval[3], allDataEval[5]])

Read 6 files


In [8]:
TOKENIZER_REPO = 'google/mt5-base'
MODEL_REPO = TOKENIZER_REPO

tokenizer, model = loadTokenizerAndSeq2SeqLM(TOKENIZER_REPO, MODEL_REPO,
                                             use_pretrained = False)

MAX_SEQ_LEN = 144
set_tokenizer_lang = False
add_lang_token = True

/home/group2/miniconda3/envs/pytorch_Tan3010/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
sampleInputSentence = 'Phiên dịch tiếng Lào: câu này sẽ được dịch thành tiếng Lào.'
sampleOutputSentence = 'ການແປພາສາລາວ: ປະໂຫຍກນີ້ຈະຖືກແປເປັນພາສາລາວ.'
toyDataset = createDataset([[sampleInputSentence]], [])

tokenizerOutput = format_translation_data(toyDataset['translation'][0],
                                          tokenizer, set_tokenizer_lang, add_lang_token, 
                                          input_lang = 'vi', target_lang = 'lo', random_lang = False,
                                          MAX_SEQ_LEN = MAX_SEQ_LEN,
                                          )
print(tokenizerOutput[0])
print(tokenizer.convert_ids_to_tokens(tokenizerOutput[0]))
print(tokenizer.decode(tokenizerOutput[0]))

model.eval()
modelOutput = model(tokenizerOutput[0].unsqueeze(0),
                    attention_mask = tokenizerOutput[2].unsqueeze(0),
                    labels = tokenizerOutput[1].unsqueeze(0))

modelGenerate = model.generate(tokenizerOutput[0].unsqueeze(0), max_new_tokens = MAX_SEQ_LEN)
print(modelGenerate)

output_text = tokenizer.decode(modelGenerate[0])
print(tokenizer.convert_ids_to_tokens(modelGenerate[0]))
print(output_text)

tensor([ 1042,   409,  1792,   878,  1792,   409,   669,   363, 12093,   331,
         3168,  2868,   370,   458,  5395,   267,  4650,   273,  2424,   259,
          263,  2254,   259,  1318,   331,  3168,   394,  3255,  2868,   370,
          458,  5395,   260,     1,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [10]:
add_lang_tokens_to_tokenizer(tokenizer, model)

In [11]:
tokenizerOutput = format_translation_data(toyDataset['translation'][0],
                                          tokenizer, set_tokenizer_lang, add_lang_token, 
                                          input_lang = 'vi', target_lang = 'lo', random_lang = False,
                                          MAX_SEQ_LEN = MAX_SEQ_LEN,
                                          )

tokens = tokenizer.convert_ids_to_tokens(tokenizerOutput[0])
print(tokens) # Make sure that the special translation token is not 'fragmented'

['<|__lo__|> ', '▁P', 'hiên', '▁d', 'ịch', '▁tiế', 'ng', '▁L', 'ào', ':', '▁câ', 'u', '▁này', '▁', 's', 'ẽ', '▁', 'được', '▁d', 'ịch', '▁th', 'ành', '▁tiế', 'ng', '▁L', 'ào', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<

In [12]:
# Testing `data_transform`
toyDataset = createDataset([[sampleInputSentence]], [[sampleOutputSentence]])
in_ids, out_ids, attention_mask = format_translation_data(toyDataset['translation'][0],
                                                          tokenizer, set_tokenizer_lang, add_lang_token, 
                                                          input_lang = 'vi', target_lang = 'lo', random_lang = False,
                                                          MAX_SEQ_LEN = MAX_SEQ_LEN,
                                                          )

print(' '.join(tokenizer.convert_ids_to_tokens(in_ids)))
print(' '.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Testing data generator
data_gen = get_data_generator(
    train_dataset,
    tokenizer, set_tokenizer_lang, add_lang_token,
    MAX_SEQ_LEN,
    input_lang = 'vi', target_lang = 'lo', random_lang = False,
)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)
print('Attention mask shape:', data_batch[2].shape)

<|__lo__|>  ▁P hiên ▁d ịch ▁tiế ng ▁L ào : ▁câ u ▁này ▁ s ẽ ▁ được ▁d ịch ▁th ành ▁tiế ng ▁L ào . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
▁ການ ແປ ພາສາ ລາວ : ▁ປະ ໂຫ ຍ ກ ນີ້ ຈະ ຖືກ ແປ ເປັນ ພາສາ ລາວ . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad

In [13]:
# Constants
n_epochs = 25
batch_size = 1

print_freq = 100
checkpoint_freq = 500

lr = 7.5e-4

n_batches = int(np.ceil(len(train_dataset) / batch_size))
total_steps = n_epochs * n_batches
n_warmup_steps = int(total_steps * 0.2)

In [14]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps
)

/home/group2/miniconda3/envs/pytorch_Tan3010/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
model_name = 'mT5Base'
n_direction = 2
data_description = 'AllCleaned'
model_checkpoint = ('%s_%dD_%s_batch%d_checkpoint.pt' 
                    % (model_name, n_direction, data_description, batch_size))
model_path = ('%s_%dD_%s_batch%d_fineTunedEpoch{}.pt' 
                    % (model_name, n_direction, data_description, batch_size))

In [16]:
input_lang = None
target_lang = None
random_lang = True

In [17]:
losses, evalLosses = train(model,
                           tokenizer, set_tokenizer_lang, add_lang_token,
                           MAX_SEQ_LEN,
                           input_lang, target_lang, random_lang,
                           batch_size,
                           optimizer, scheduler,
                           n_epochs, n_batches,
                           train_dataset, eval_dataset,
                           print_freq, checkpoint_freq,
                           model_checkpoint, model_path)

  0%|          | 0/76500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plotLoss(losses, smoothing_window_size = 50)

In [ ]:
plotLoss(evalLosses, smoothing_window_size = 1)

In [ ]:
eval_model(model, eval_dataset)